# Done code Vn Express

In [13]:
import requests
from bs4 import BeautifulSoup

url = 'https://vnexpress.net'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

menu_items = soup.find_all('li', attrs={'data-id': True})

categories = {}

for li in menu_items:
    # Tìm thẻ <a> đầu tiên trong thẻ <li> hiện tại
    a_tags = li.find_all('a', recursive=False)
    if not a_tags:
        continue
    main_a = a_tags[0]
    main_name = main_a.get_text(strip=True)
    main_link = main_a['href']
    if main_link.startswith('/'):
        main_link = 'https://vnexpress.net' + main_link

    categories[main_name] = {
        'link': main_link,
        'subcategories': {}
    }

    # Tìm thẻ <ul> con trong thẻ <li> hiện tại
    sub_menu = li.find('ul')
    if sub_menu:
        sub_items = sub_menu.find_all('a')
        for sub_a in sub_items:
            sub_name = sub_a.get_text(strip=True)
            sub_link = sub_a['href']
            if sub_link.startswith('/'):
                sub_link = 'https://vnexpress.net' + sub_link
            categories[main_name]['subcategories'][sub_name] = sub_link

# In kết quả
print(f"📚 Tìm thấy {len(categories)} chuyên mục chính:")
for main_name, info in categories.items():
    print(f"- {main_name}: {info['link']}")
    if info['subcategories']:
        for sub_name, sub_link in info['subcategories'].items():
            print(f"    ▪ {sub_name}: {sub_link}")


📚 Tìm thấy 19 chuyên mục chính:
- Thời sự: https://vnexpress.net/thoi-su
- Thế giới: https://vnexpress.net/the-gioi
- Kinh doanh: https://vnexpress.net/kinh-doanh
- Công nghệ: https://vnexpress.net/cong-nghe
- Khoa học: https://vnexpress.net/khoa-hoc
- Video: https://video.vnexpress.net
- Podcasts: https://vnexpress.net/podcast
- Góc nhìn: https://vnexpress.net/goc-nhin
- Bất động sản: https://vnexpress.net/bat-dong-san
- Sức khỏe: https://vnexpress.net/suc-khoe
- Thể thao: https://vnexpress.net/the-thao
- Giải trí: https://vnexpress.net/giai-tri
- Pháp luật: https://vnexpress.net/phap-luat
- Giáo dục: https://vnexpress.net/giao-duc
- Đời sống: https://vnexpress.net/doi-song
- Xe: https://vnexpress.net/oto-xe-may
- Du lịch: https://vnexpress.net/du-lich
- Ý kiến: https://vnexpress.net/y-kien
- Tâm sự: https://vnexpress.net/tam-su


In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

data = []

url = 'https://vnexpress.net'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# Tìm tất cả <li> có thuộc tính data-id (chuyên mục)
menu_items = soup.find_all('li', attrs={'data-id': True})

categories = {}

for li in menu_items:
    a_tag = li.find('a')
    if a_tag and 'href' in a_tag.attrs:
        name = a_tag.get_text(strip=True)
        link = a_tag['href']
        if link.startswith('/'):  # nối với domain
            link = 'https://vnexpress.net' + link
        categories[name] = link

# In ra các chuyên mục
print(f"📚 Tìm thấy {len(categories)} chuyên mục:")
for name, link in categories.items():
    print(f"- {name}: {link}")

article_counter = 1  # ✅ Khai báo biến đếm ID bài viết ở đây

# Bước 2: Crawl mỗi chuyên mục 5 trang
num_pages = 10

for category, base_url in categories.items():
    print(f"\n=== 📂 Crawling chuyên mục: {category.upper()} ===")
    
    for page in range(1, num_pages + 1):
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}-p{page}"
        
        print(f"🔎 Trang {page}: {url}")
        
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            articles = soup.find_all('article', class_='item-news')
        except:
            print("⚠️ Không thể load trang.")
            continue

        for article in articles:
            title_tag = article.find('h3', class_='title-news')
            if title_tag:
                title = title_tag.get_text(strip=True)
                link = title_tag.find('a')['href']
                print(f"📰 {title}")
                
                try:
                    article_res = requests.get(link)
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')

                    content_div = article_soup.find('article', class_='fck_detail')
                    if content_div:
                        paragraphs = content_div.find_all('p')
                        content_text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
                    else:
                        content_text = None

                    time_tag = article_soup.find('span', class_='date')
                    date = time_tag.get_text(strip=True) if time_tag else None

                    
                    author = None
                    normal_paragraphs = article_soup.find_all('p', class_='Normal')
                    for p in reversed(normal_paragraphs):  # Duyệt ngược từ cuối lên
                        strong_tag = p.find('strong')
                        if strong_tag:
                            author = strong_tag.get_text(strip=True)
                            break

                    # Tạo ID bài viết theo định dạng "VNEX_01", "VNEX_02", ...
                    article_id = f"VNEX_{article_counter:02d}"
                    article_counter += 1
                    
                    data.append({
                        'id': article_id,
                        'title': title,
                        'link': link,
                        'content': content_text,
                        'date': date,
                        'author': author,
                        'category': category,
                        'source': 'VN Express'
                    })

                except Exception as e:
                    print(f"❌ Lỗi bài viết: {e}")
                
                time.sleep(1)

# Bước 3: Tạo DataFrame
df = pd.DataFrame(data)
print(df.head())

# Tuỳ chọn: lưu CSV
# df.to_csv('vnexpress_all_categories.csv', index=False, encoding='utf-8-sig')

📚 Tìm thấy 19 chuyên mục:
- Thời sự: https://vnexpress.net/thoi-su
- Thế giới: https://vnexpress.net/the-gioi
- Kinh doanh: https://vnexpress.net/kinh-doanh
- Công nghệ: https://vnexpress.net/cong-nghe
- Khoa học: https://vnexpress.net/khoa-hoc
- Video: https://video.vnexpress.net
- Podcasts: https://vnexpress.net/podcast
- Góc nhìn: https://vnexpress.net/goc-nhin
- Bất động sản: https://vnexpress.net/bat-dong-san
- Sức khỏe: https://vnexpress.net/suc-khoe
- Thể thao: https://vnexpress.net/the-thao
- Giải trí: https://vnexpress.net/giai-tri
- Pháp luật: https://vnexpress.net/phap-luat
- Giáo dục: https://vnexpress.net/giao-duc
- Đời sống: https://vnexpress.net/doi-song
- Xe: https://vnexpress.net/oto-xe-may
- Du lịch: https://vnexpress.net/du-lich
- Ý kiến: https://vnexpress.net/y-kien
- Tâm sự: https://vnexpress.net/tam-su

=== 📂 Crawling chuyên mục: THỜI SỰ ===
🔎 Trang 1: https://vnexpress.net/thoi-su
📰 Cao tốc qua Bà Rịa - Vũng Tàu thông xe kỹ thuật ngày 19/4
📰 Gia tộc Minh Tơ 100 n

In [46]:
df

,id,title,link,content,date,author,category,source
0,VNEX_01,Cao tốc qua Bà Rịa - Vũng Tàu thông xe kỹ thuậ...,https://vnexpress.net/cao-toc-qua-ba-ria-vung-...,Thông tin được UBND tỉnh Bà Rịa - Vũng Tàu nêu...,"Thứ ba, 15/4/2025, 00:00 (GMT+7)",Trường Hà,Thời sự,VN Express
1,VNEX_02,Gia tộc Minh Tơ 100 năm giữ hồn tuồng cổ Sài Gòn,https://vnexpress.net/gia-toc-minh-to-100-nam-...,None,None,None,Thời sự,VN Express
2,VNEX_03,Lập sở chỉ huy tiền phương hỗ trợ y tế dịp đại...,https://vnexpress.net/lap-so-chi-huy-tien-phuo...,Thứ trưởng Y tế Nguyễn Tri Thức nói như trên n...,"Thứ hai, 14/4/2025, 15:36 (GMT+7)",Lê Phương,Thời sự,VN Express
3,VNEX_04,Những thương hiệu quốc dân 'vang bóng một thời',https://vnexpress.net/nhung-thuong-hieu-quoc-d...,Xà bông Cô Balà nhãn hiệu do ông Trương Văn Bề...,"Thứ hai, 14/4/2025, 11:00 (GMT+7)",Phương Đông - Quỳnh Trần,Thời sự,VN Express
4,VNEX_05,'Đầu tàu' TP HCM trước cuộc chuyển đổi kép,https://vnexpress.net/dau-tau-tp-hcm-truoc-cuo...,None,None,None,Thời sự,VN Express
...,...,...,...,...,...,...,...,...
3051,VNEX_3052,Cảm thấy có lỗi sau mỗi lần la mắng con,https://vnexpress.net/cam-thay-co-loi-sau-moi-...,"Vợ chồng tôi trên 40 tuổi, con trai học lớp 8....","Thứ ba, 4/3/2025, 16:00 (GMT+7)",Thùy Linh,Tâm sự,VN Express
3052,VNEX_3053,Yêu người nhiều ưu điểm nhưng không muốn rời x...,https://vnexpress.net/yeu-nguoi-nhieu-uu-diem-...,"Tôi 30 tuổi, lúc trước du học Australia, sau đ...","Thứ hai, 3/3/2025, 20:00 (GMT+7)",Duyên Hải,Tâm sự,VN Express
3053,VNEX_3054,"Sống chung 3 năm, tôi hết chịu nổi chồng",https://vnexpress.net/song-chung-3-nam-toi-het...,"Tôi 45 tuổi, từng ly hôn, đang chung sống với ...","Thứ hai, 3/3/2025, 18:00 (GMT+7)",Huyền Nga,Tâm sự,VN Express
3054,VNEX_3055,"Lúc chờ kết quả khám của bố, tôi hiểu cần quan...",https://vnexpress.net/luc-cho-ket-qua-kham-cua...,"Tôi 37 tuổi, buôn bán nhỏ, nhà có ba anh chị e...","Thứ hai, 3/3/2025, 16:00 (GMT+7)",Hoàng Thi,Tâm sự,VN Express


# Done Tienphong.vn

In [47]:

url = 'https://tienphong.vn/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')  # Phân tích nội dung HTML

# Tìm tất cả các mục menu chuyên mục (có thuộc tính data-id)
menu_items = soup.find_all('li', attrs={'data-id': True})

categories = {}

# Duyệt qua từng mục menu để lấy tên chuyên mục và link tương ứng
for li in menu_items:
    a_tag = li.find('a')
    if a_tag and 'href' in a_tag.attrs:
        name = a_tag.get_text(strip=True)  # Tên chuyên mục
        link = a_tag['href']  # Link tương ứng
        if link.startswith('/'):  # Nếu là link tương đối thì nối với domain chính
            link = 'https://tienphong.vn/' + link
        categories[name] = link

# In ra tất cả các chuyên mục đã lấy được
print(f"📚 Tìm thấy {len(categories)} chuyên mục:")
for name, link in categories.items():
    print(f"- {name}: {link}")
    


📚 Tìm thấy 85 chuyên mục:
- Xã hội: https://tienphong.vn/xa-hoi/
- Chính trị: https://tienphong.vn/chinh-tri/
- Tin tức: https://tienphong.vn/hh-tin-tuc/
- Phóng sự: https://tienphong.vn/xa-hoi-phong-su/
- Kinh tế: https://tienphong.vn/kinh-te/
- Thị trường: https://tienphong.vn/video-thi-truong/
- Doanh nghiệp: https://tienphong.vn/doanh-nghiep/
- Đầu tư: https://tienphong.vn/song-xanh-dau-tu/
- Tài chính - Chứng khoán: https://tienphong.vn/tai-chinh-chung-khoan/
- Sóng xanh: https://tienphong.vn/song-xanh/
- Xanh 4.0: https://tienphong.vn/xanh-4-0/
- Lifestyle: https://tienphong.vn/lifestyle/
- Net zero: https://tienphong.vn/netzero/
- Di chuyển xanh: https://tienphong.vn/di-chuyen-xanh/
- Địa ốc: https://tienphong.vn/dia-oc/
- Đô thị - Dự án: https://tienphong.vn/do-thi-du-an/
- Thị trường - Doanh nghiệp: https://tienphong.vn/thi-truong-doanh-nghiep/
- Nhà đẹp - Kiến trúc: https://tienphong.vn/nha-dep-kien-truc/
- Chuyên gia - Tư vấn: https://tienphong.vn/chuyen-gia-tu-van/
- Media 

In [48]:
article_counter = 1
seen_links = set()  # ✅ Lưu các link đã crawl để tránh trùng


num_pages = 2

for category, base_url in categories.items():
    print(f"\n=== 📂 Crawling chuyên mục: {category.upper()} ===")
    
    for page in range(1, num_pages + 1):
        url = base_url if page == 1 else f"{base_url}-p{page}"
        print(f"🔎 Trang {page}: {url}")
        
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            articles = soup.find_all('article', class_='story')
        except:
            print("⚠️ Không thể load trang.")
            continue

        for article in articles:
            # ✅ Gộp tìm cả h2 và h3
            heading_tag = article.find(['h2', 'h3'], class_='story__heading')
            link_tag = heading_tag.find('a', class_='cms-link') if heading_tag else None

            if not link_tag:
                continue

            link = link_tag['href']
            if link in seen_links:  # ✅ Kiểm tra trùng
                continue
            seen_links.add(link)

            title = link_tag.get_text(strip=True)
            print(f"📰 {title}")

            try:
                article_res = requests.get(link)
                article_soup = BeautifulSoup(article_res.text, 'html.parser')

                content_div = article_soup.find('div', class_='col-27 article-content')
                if content_div:
                    paragraphs = content_div.find_all('p')
                    content_text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
                else:
                    content_text = None

                time_tag = article_soup.find('span', class_='time')
                date = time_tag.get_text(strip=True) if time_tag else None

                author = None
                author_tag_1 = article_soup.find('div', class_='article__author')
                if author_tag_1:
                    name_tag = author_tag_1.find('span', class_='name cms-author')
                    if name_tag:
                        author = name_tag.get_text(strip=True)

                if not author:
                    author_tag_2 = article_soup.find('span', class_='author')
                    if author_tag_2:
                        cms_author = author_tag_2.find('a', class_='cms-author')
                        if cms_author:
                            name_span = cms_author.find('span')
                            if name_span:
                                author = name_span.get_text(strip=True)

                article_id = f"TP_{article_counter:02d}"
                article_counter += 1

                data.append({
                    'id': article_id,
                    'title': title,
                    'link': link,
                    'content': content_text,
                    'date': date,
                    'author': author,
                    'category': category,
                    'source': 'Tiền Phong'
                })

            except Exception as e:
                print(f"❌ Lỗi bài viết: {e}")
            
            time.sleep(2)

df = pd.DataFrame(data)
print(df.head())



=== 📂 Crawling chuyên mục: XÃ HỘI ===
🔎 Trang 1: https://tienphong.vn/xa-hoi/
📰 Tổng Bí thư Tô Lâm: Sắp xếp, tinh gọn tổ chức bộ máy 'rất đúng, rất trúng, hợp ý Đảng, lòng dân'
📰 Hải Phòng, Hải Dương sắp xếp cho cán bộ làm việc sau sáp nhập
📰 Phú Thọ, Vĩnh Phúc, Hòa Bình lập ban chỉ đạo sáp nhập tỉnh
📰 Mưa dông mạnh bất thường ở Đắk Nông
📰 Dự án cầu Đuống mới nguy cơ dừng thi công do Hà Nội chậm mặt bằng
📰 Thủ tướng Phạm Minh Chính đề nghị Trung Quốc dành ưu tiên cao cho hợp tác đường sắt
📰 Chủ tịch Quốc hội: Dự kiến lấy ý kiến nhân dân về sửa đổi Hiến pháp trong 1 tháng
📰 Dự kiến tên gọi và trung tâm hành chính của 34 tỉnh thành sau sáp nhập
📰 Toàn cảnh lễ đón Tổng Bí thư, Chủ tịch nước Trung Quốc Tập Cận Bình tại sân bay Nội Bài
📰 Đoàn xe chở Tổng Bí thư, Chủ tịch nước Trung Quốc Tập Cận Bình trên phố Hà Nội
📰 TIN NHANH: Dự báo Hà Nội sắp nóng 40 độ C; Thành phố bị sét đánh 6.000 lần một đêm
📰 Thí sinh Hoa hậu Việt Nam nhận xét về nhau
📰 Thanh niên Trung Quốc khám phá bản sắc văn hó

In [49]:
df

,id,title,link,content,date,author,category,source
0,VNEX_01,Cao tốc qua Bà Rịa - Vũng Tàu thông xe kỹ thuậ...,https://vnexpress.net/cao-toc-qua-ba-ria-vung-...,Thông tin được UBND tỉnh Bà Rịa - Vũng Tàu nêu...,"Thứ ba, 15/4/2025, 00:00 (GMT+7)",Trường Hà,Thời sự,VN Express
1,VNEX_02,Gia tộc Minh Tơ 100 năm giữ hồn tuồng cổ Sài Gòn,https://vnexpress.net/gia-toc-minh-to-100-nam-...,None,None,None,Thời sự,VN Express
2,VNEX_03,Lập sở chỉ huy tiền phương hỗ trợ y tế dịp đại...,https://vnexpress.net/lap-so-chi-huy-tien-phuo...,Thứ trưởng Y tế Nguyễn Tri Thức nói như trên n...,"Thứ hai, 14/4/2025, 15:36 (GMT+7)",Lê Phương,Thời sự,VN Express
3,VNEX_04,Những thương hiệu quốc dân 'vang bóng một thời',https://vnexpress.net/nhung-thuong-hieu-quoc-d...,Xà bông Cô Balà nhãn hiệu do ông Trương Văn Bề...,"Thứ hai, 14/4/2025, 11:00 (GMT+7)",Phương Đông - Quỳnh Trần,Thời sự,VN Express
4,VNEX_05,'Đầu tàu' TP HCM trước cuộc chuyển đổi kép,https://vnexpress.net/dau-tau-tp-hcm-truoc-cuo...,None,None,None,Thời sự,VN Express
...,...,...,...,...,...,...,...,...
5692,TP_2637,Cảnh giác trước trang page giả mạo Tiền Phong ...,https://tienphong.vn/canh-giac-truoc-trang-pag...,"Trong nhiều năm,Giải Vô địch Quốc gia Marathon...",09/11/2024 | 19:30,Thanh Hải,20 năm TPO,Tiền Phong
5693,TP_2638,Tiền Phong Golf Championship 2024: Tiếp thêm n...,https://tienphong.vn/tien-phong-golf-champions...,"Ngày 2/11, Tiền PhongGolfChampionship - Vì Tài...",02/11/2024 | 12:29,Hà Trang,20 năm TPO,Tiền Phong
5694,TP_2639,Ra mắt chuyên mục Sóng Xanh trên Báo điện tử T...,https://tienphong.vn/ra-mat-chuyen-muc-song-xa...,Cuộc cách mạng công nghiệp lần thứ tư đang tác...,01/11/2024 | 07:07,Tiền Phong,20 năm TPO,Tiền Phong
5695,TP_2640,"Tiền Phong Golf Championship: Các nhà vô địch,...",https://tienphong.vn/tien-phong-golf-champions...,Nhà vô địch 2018: NguyễnBảo Long\nKhông chỉ qu...,29/10/2024 | 08:38,THANH HẢI,20 năm TPO,Tiền Phong


In [51]:
df.to_csv('/Users/doanhtuan137/Desktop/_articles.csv', index=False)
print("Data has been saved to '_articles.csv'")

Data has been saved to '_articles.csv'
